In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup #library that allows to easily perform web scraping, no copy pasting, programatically grabs a blog post from web
import requests #helps to calls out blog post & bring back the result, then pass them to beautiful soup

#1. Load Summarization Pipeline

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


#2. Get Blog Post

In [ ]:
URL = "https://en.wikipedia.org/wiki/Ananda_College"

In [ ]:
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [ ]:
r = requests.get(URL) #grab entire webpaage

In [ ]:
r

<Response [200]>

In [ ]:
r.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-night-mode-clientpref-0 vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Ananda College - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled 

In [ ]:
#extract blog posts from the webpage
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p']) #h1 -headings   p - paragraphs
text = [result.text for result in results] #only texts, no tags
ARTICLE = ' '.join(text) #one big block of articles

In [ ]:
results

[<h1 class="firstHeading mw-first-heading" id="firstHeading"><span class="mw-page-title-main">Ananda College</span></h1>,
 <p class="mw-empty-elt">
 </p>,
 <p><b>Ananda College</b> (<a href="/wiki/Sinhala_language" title="Sinhala language">Sinhala</a>: <span lang="si">ආනන්ද විද්‍යාලය</span>) is a prestigious Buddhist school in <a href="/wiki/Maradana" title="Maradana">Maradana</a>, <a href="/wiki/Colombo" title="Colombo">Colombo</a>. It was established as the English Buddhist School by Colonel <a href="/wiki/Henry_Steel_Olcott" title="Henry Steel Olcott">Henry Steel Olcott</a> in 1886. In the present day, it provides primary and secondary education on a campus of 15 acres (61,000 m<sup>2</sup>).<sup class="reference" id="cite_ref-foundn_1-0"><a href="#cite_note-foundn-1">[1]</a></sup>
 </p>,
 <p>Following a meeting of Buddhists at <a href="/wiki/Pettah,_Sri_Lanka" title="Pettah, Sri Lanka">Pettah</a>, under the patronage of <a href="/wiki/Hikkaduwe_Sri_Sumangala_Thera" title="Hikkaduwe

In [ ]:
text

['Ananda College',
 '\n',
 'Ananda College (Sinhala: ආනන්ද විද්\u200dයාලය) is a prestigious Buddhist school in Maradana, Colombo. It was established as the English Buddhist School by Colonel Henry Steel Olcott in 1886. In the present day, it provides primary and secondary education on a campus of 15 acres (61,000\xa0m2).[1]\n',
 'Following a meeting of Buddhists at Pettah, under the patronage of Hikkaduwe Sri Sumangala Thera, an English-Buddhist school was inaugurated at 19 Prince Street on 1 November 1886 by the Buddhist Theosophical Society. The first session was attended by 37 students. In 1888, when about 130 boys were attending, it moved to 61 Maliban Street. C. W. Leadbeater was appointed the first principal of Ananda today.[2]\n',
 'By the time the school was officially registered in March 1889, there were 120 students. That same year, J. P. R. Weerasuriya became the first Anandian to pass the Cambridge junior examination. The Cambridge graduate and confessed Buddhist A. E. Bult

In [ ]:
ARTICLE

'Ananda College \n Ananda College (Sinhala: ආනන්ද විද්\u200dයාලය) is a prestigious Buddhist school in Maradana, Colombo. It was established as the English Buddhist School by Colonel Henry Steel Olcott in 1886. In the present day, it provides primary and secondary education on a campus of 15 acres (61,000\xa0m2).[1]\n Following a meeting of Buddhists at Pettah, under the patronage of Hikkaduwe Sri Sumangala Thera, an English-Buddhist school was inaugurated at 19 Prince Street on 1 November 1886 by the Buddhist Theosophical Society. The first session was attended by 37 students. In 1888, when about 130 boys were attending, it moved to 61 Maliban Street. C. W. Leadbeater was appointed the first principal of Ananda today.[2]\n By the time the school was officially registered in March 1889, there were 120 students. That same year, J. P. R. Weerasuriya became the first Anandian to pass the Cambridge junior examination. The Cambridge graduate and confessed Buddhist A. E. Bultjens became princ

#3. Chunk Text


In [ ]:
max_chunk = 500
#because there is a limitation on how much text can be passed through baseline summarization pipeline
#initially splitup article into individual sentences

In [ ]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [ ]:
sentences = ARTICLE.split('<eos>')

In [ ]:
sentences[20]

' Tudor Rajapaksha donated 3.'

In [ ]:
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: #check if there is a chunk in chunks list
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk: # if there is a chunk in chunks list add next sentence to it and check the length
            chunks[current_chunk].extend(sentence.split(' ')) # if length doesnt exceed max limit, add next sentence to current chunk
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)  #if there is no chunk in chunk list,add splitted sentence to  chunk
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)): #add back words to completed sentences
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
chunks[0]

"Ananda College \n Ananda College (Sinhala: ආනන්ද විද්\u200dයාලය) is a prestigious Buddhist school in Maradana, Colombo.  It was established as the English Buddhist School by Colonel Henry Steel Olcott in 1886.  In the present day, it provides primary and secondary education on a campus of 15 acres (61,000\xa0m2). [1]\n Following a meeting of Buddhists at Pettah, under the patronage of Hikkaduwe Sri Sumangala Thera, an English-Buddhist school was inaugurated at 19 Prince Street on 1 November 1886 by the Buddhist Theosophical Society.  The first session was attended by 37 students.  In 1888, when about 130 boys were attending, it moved to 61 Maliban Street.  C.  W.  Leadbeater was appointed the first principal of Ananda today. [2]\n By the time the school was officially registered in March 1889, there were 120 students.  That same year, J.  P.  R.  Weerasuriya became the first Anandian to pass the Cambridge junior examination.  The Cambridge graduate and confessed Buddhist A.  E.  Bultj

In [ ]:
len(chunks)

2

In [ ]:
len(chunks[1].split(' '))

325

#4. Summarize Text

In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
res

[{'summary_text': ' Ananda College is a prestigious Buddhist school in Maradana, Colombo . It was established as the English Buddhist School by Colonel Henry Steel Olcott in 1886 . It provides primary and secondary education on a campus of 15 acres (61,000\xa0m2)'},
 {'summary_text': ' The Ananda College war memorial is situated in front of the Henry Steel Olcott Hall and is dedicated to alumni of Ananda college who died while members of the Sri Lankan armed forces . Ananda Daham Pasala (Ananda Dhamma School) is the Sunday school . The Olcott oration is an annual event .'}]

In [ ]:
' '.join([summ['summary_text'] for summ in res])

' Ananda College is a prestigious Buddhist school in Maradana, Colombo . It was established as the English Buddhist School by Colonel Henry Steel Olcott in 1886 . It provides primary and secondary education on a campus of 15 acres (61,000\xa0m2)  The Ananda College war memorial is situated in front of the Henry Steel Olcott Hall and is dedicated to alumni of Ananda college who died while members of the Sri Lankan armed forces . Ananda Daham Pasala (Ananda Dhamma School) is the Sunday school . The Olcott oration is an annual event .'

#5.Output text file

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/summaryhuggingface.txt'
with open(file_path,'w') as f:
  f.write(text)